<a href="https://colab.research.google.com/github/Dolphine12/AI_Ethics/blob/main/Racial_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing

# Upload CSV
uploaded = files.upload()

# Load data
df = pd.read_csv("compass_data.csv")

# Inspect your data
print(df.columns)
print(df['race'].unique())  # Check what values are in 'race'

# 🧹 Clean and encode race
# You might need to update this mapping based on actual values in your dataset
race_map = {
    'African-American': 0,
    'Caucasian': 1
}
df = df[df['race'].isin(race_map.keys())]  # Keep only these 2 races
df['race'] = df['race'].map(race_map)

# Drop rows with missing values in important fields
df = df.dropna(subset=['race', 'two_year_recid'])

# Now convert to AIF360 StandardDataset
dataset = StandardDataset(df,
                          label_name='two_year_recid',
                          favorable_classes=[0],  # Not reoffended
                          protected_attribute_names=['race'],
                          privileged_classes=[[1]],  # White
                          features_to_drop=['id', 'name'] if 'id' in df.columns and 'name' in df.columns else [])

# Define groups
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

# Baseline fairness check
metric_orig = BinaryLabelDatasetMetric(dataset,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("Disparate impact (original):", metric_orig.disparate_impact())

# Split dataset
train, test = dataset.split([0.7], shuffle=True)

# Continue only if data is available
if train.features.shape[0] == 0:
    raise ValueError("No training data available after cleaning. Check your dataset.")

# Train model
scaler = StandardScaler()
X_train = scaler.fit_transform(train.features)
y_train = train.labels.ravel()

model = LogisticRegression()
model.fit(X_train, y_train)

# Predict
X_test = scaler.transform(test.features)
y_pred = model.predict(X_test)
test_pred = test.copy()
test_pred.labels = y_pred

# Fairness metric
metric_pred = ClassificationMetric(test, test_pred,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)

# Visualize false positive rate
fpr_black = metric_pred.false_positive_rate(unprivileged=True)
fpr_white = metric_pred.false_positive_rate(privileged=True)

plt.bar(['Black', 'White'], [fpr_black, fpr_white], color=['red', 'blue'])
plt.title('False Positive Rate by Race')
plt.ylabel('Rate')
plt.show()

# Remediation
RW = Reweighing(unprivileged_groups, privileged_groups)
train_transf = RW.fit_transform(train)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 4.9 MB/s eta 0:00:00


pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


# **Analyzing Racial Bias in the COMPAS Recidivism Dataset Using AI Fairness 360**

-The **COMPAS dataset** is widely used to assess the fairness of algorithmic risk scores in criminal justice. In this study, we used IBM's AI Fairness 360 toolkit to investigate potential racial bias in the dataset’s risk predictions, particularly focusing on disparities between Black and White individuals.

-Our baseline analysis used:
1.  logistic regression to predict recidivism, and fairness metrics were computed using the AIF360 library. The initial results revealed significant bias: the false positive rate (FPR) for Black individuals was considerably higher than for White individuals, indicating that Black individuals were more frequently misclassified as high-risk when they did not reoffend, highlighting a key fairness concern.

- Visualization of the FPR by race clearly illustrated this disparity, reinforcing concerns raised by prior research and media investigations into the use of COMPAS scores.

To address this bias, we applied the Reweighing algorithm, a preprocessing technique that adjusts instance weights in the training data to mitigate bias while preserving the original labels. After reweighing, the disparate impact metric improved substantially, and the difference in FPR across racial groups was reduced. However, slight trade-offs in overall model accuracy were observed, a common consequence of fairness optimization.

In conclusion, our analysis confirmed the presence of racial bias in the COMPAS dataset, particularly in the form of false positive disparities. Remediation via reweighing proved partially effective in reducing this unfairness. Future work could explore more advanced mitigation techniques like adversarial debiasing or post-processing methods to further improve fairness without compromising model performance. Ultimately, fairness-aware tools like AIF360 are essential for building transparent and accountable AI systems in sensitive domains such as criminal justice.